## 미세먼지 알리미 텔레그램 봇

In [1]:
import telegram
import xml.etree.ElementTree as et
import json
import requests as req

In [2]:
# data.go.kr에서 제공하는 미세먼지 API
url1 = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty?serviceKey=O1nHnnvHdTlHJn9IB55OEVZsW8tvqkALFXNRBo9nGXjODXETiIA9SjoOZAyLAE3OyADFq%2BmhF36yldlCJppS2A%3D%3D&returnType=xml&numOfRows=10000&pageNo=1&sidoName=%EC%A0%84%EA%B5%AD&ver=1.0'
url2 = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty?serviceKey=O1nHnnvHdTlHJn9IB55OEVZsW8tvqkALFXNRBo9nGXjODXETiIA9SjoOZAyLAE3OyADFq%2BmhF36yldlCJppS2A%3D%3D&returnType=json&numOfRows=10000&pageNo=1&sidoName=%EC%A0%84%EA%B5%AD&ver=1.0'

## xml 형태로 불러오기

In [3]:
res = req.get(url1)
res.text[:500]

'<?xml version="1.0" encoding="UTF-8"?>\r\n<response>\n  <header>\n    <resultCode>00</resultCode>\n    <resultMsg>NORMAL_CODE</resultMsg>\n  </header>\n  <body>\n    <items>\n      <item>\n        <so2Grade/>\n        <coFlag/>\n        <khaiValue>48</khaiValue>\n        <so2Value>-</so2Value>\n        <coValue>0.4</coValue>\n        <pm25Flag/>\n        <pm10Flag/>\n        <o3Grade>1</o3Grade>\n        <pm10Value>17</pm10Value>\n        <khaiGrade>1</khaiGrade>\n        <pm25Value>6</pm25Value>\n        <sidoName>'

In [4]:
# 서버에서 받아온 데이터를 파일로 저장
with open('../data/air.xml', 'w', encoding='UTF-8') as f:
    f.writelines(res.text)

In [5]:
# xml 형태로 저장된 파일에서 원하는 정보 얻기
tree = et.parse('../data/air.xml')
root = tree.getroot()

In [6]:
# 미세정보 : 작성일자, 시도, 구/동/읍/면, 미세먼지/초미세먼지농도
cnt = 0
for data in root.findall('body/items/item'):
    date = data.find('dataTime').text
    sido = data.find('sidoName').text
    stnm = data.find('stationName').text
    pm10 = data.find('pm10Value').text
    pm25 = data.find('pm25Value').text
    
    if cnt <= 10:    # 결과 중 1건만 출력
        print(date, sido, stnm, pm10, pm25)
        cnt = cnt + 1
    else: break    

2022-09-01 12:00 세종 한솔동 17 6
2022-09-01 12:00 세종 부강면 18 5
2022-09-01 12:00 세종 보람동 - -
2022-09-01 13:00 강원 중앙동(강원) 16 11
2022-09-01 13:00 강원 반곡동(명륜동) 24 7
2022-09-01 13:00 강원 문막읍 7 2
2022-09-01 13:00 강원 지정면 9 4
2022-09-01 13:00 강원 주문진읍 15 12
2022-09-01 13:00 강원 옥천동 15 3
2022-09-01 13:00 강원 천곡동 13 4
2022-09-01 13:00 강원 남양동1 4 -


In [7]:
# 시도, 구이름으로 원하는 지역의 미세먼지 정보 추출
sido = '서울'
gu = '구로구'

for data in root.findall('body/items/item'):
    sd = data.find('sidoName').text
    stnm = data.find('stationName').text
    
    if (sd == sido) and (stnm == gu):
        date = data.find('dataTime').text
        pm10 = data.find('pm10Value').text
        pm25 = data.find('pm25Value').text
    
        print(date, sido, stnm, pm10, pm25)
        break

2022-09-01 13:00 서울 구로구 21 7


## 미세먼지 농도별 등급
#### <img src='../img/pm10pm25.png' width='85%'>

In [8]:
def rate_pm10(val):
    if (val == '-') or (val == None): rate = '-'
    else: 
        val = int(val)
        rate = '😱'
        if val <= 30: rate = '😍'
        elif val <= 80: rate = '😊'
        elif val <= 150: rate = '🙁️'
    return rate

In [9]:
def rate_pm25(val):
    if (val == '-') or (val == None): rate = '-'
    else: 
        val = int(val)
        rate = '😱'
        if val <= 15: rate = '😍'
        elif val <= 35: rate = '😊'
        elif val <= 75: rate = '🙁️'
    return rate

In [10]:
# 대기정보를 이모지와 함께 출력
sido = '서울'
gu = '구로구'

for data in root.findall('body/items/item'):
    sd = data.find('sidoName').text
    stnm = data.find('stationName').text
    
    if (sd == sido) and (stnm == gu):
        date = data.find('dataTime').text
        pm10 = data.find('pm10Value').text
        pm25 = data.find('pm25Value').text
        p10 = rate_pm10( pm10 )
        p25 = rate_pm25( pm25 )
    
        print(date, sido, stnm, pm10, p10, pm25, p25)
        break

2022-09-01 13:00 서울 구로구 21 😍 7 😍


## json 형태로 불러오기

In [11]:
res = req.get(url2)
res.text[:500]

'{"response":{"body":{"totalCount":631,"items":[{"so2Grade":null,"coFlag":null,"khaiValue":"48","so2Value":"-","coValue":"0.4","pm25Flag":null,"pm10Flag":null,"o3Grade":"1","pm10Value":"17","khaiGrade":"1","pm25Value":"6","sidoName":"세종","no2Flag":null,"no2Grade":"1","o3Flag":null,"pm25Grade":"1","so2Flag":"자료이상","dataTime":"2022-09-01 12:00","coGrade":"1","no2Value":"0.011","stationName":"한솔동","pm10Grade":"1","o3Value":"0.029"},{"so2Grade":"1","coFlag":null,"khaiValue":"45","so2Value":"0.003","c'

In [12]:
with open('../data/air.json', 'w', encoding='UTF-8') as f:
    f.writelines(res.text)

In [13]:
with open('../data/air.json', encoding='UTF-8') as f:
    airdata = json.load(f)

In [14]:
sido = '서울'
gu = '구로구'

for item in airdata['response']['body']['items']:
    sd = item['sidoName']
    stnm = item['stationName']
    
    if (sd == sido) and (gu == stnm):
        date = item['dataTime']    
        pm10 = item['pm10Value']
        pm25 = item['pm25Value']
        p10 = rate_pm10( pm10 )
        p25 = rate_pm25( pm25 )
        print(date, sido, stnm, pm10, p10, pm25, p25)
        break

2022-09-01 13:00 서울 구로구 21 😍 7 😍


## 미세먼지 챗봇 구현하기 (json)

In [15]:
# data.go.kr에서 대기정보 받아서 파일로 저장
def reload_air():
    res = req.get(url2)
    with open('../data/air.json', 'w', encoding='UTF-8') as f:
        f.writelines(res.text)

In [16]:
# 특정지역의 대기정보와 이모지 받아오기
def fetch_air(sido, gu):
    with open('../data/air.json', encoding='UTF-8') as f:
        airdata = json.load(f)
    
    for item in airdata['response']['body']['items']:
        sd = item['sidoName']
        stnm = item['stationName']

        airinfo = ''
        if (sd == sido) and (gu == stnm):
            date = item['dataTime']    
            pm10 = item['pm10Value']
            pm25 = item['pm25Value']
            p10 = rate_pm10( pm10 )
            p25 = rate_pm25( pm25 )
            airinfo = f'{date}, {sido}, {stnm}, {pm10}, {p10}, {pm25}, {p25}'
            break
        
    return airinfo

In [17]:
fetch_air('서울','구로구')

'2022-09-01 13:00, 서울, 구로구, 21, 😍, 7, 😍'

In [18]:
from telegram.ext import Updater
from telegram.ext import MessageHandler
from telegram.ext import Filters
from telegram.ext import CommandHandler

In [19]:
token = '5786843226:'
bot = telegram.Bot(token=token)

In [20]:
# 메세지 처리 함수 정의
def air_message(update, context):
    info = update.message.text.split(',')
    airinfo = fetch_air(info[0].strip(), info[1].strip())
    update.message.reply_text(text=f'{airinfo}')
    
def reload_message(update, context):
    reload_air()
    update.message.reply_text(text=f'대기정보 갱신됨!!')

In [21]:
# 챗봇 서버 준비
updater = Updater(token)

reload_handler = CommandHandler('reload', reload_message)
air_handler = MessageHandler(Filters.text, air_message)

In [22]:
updater.dispatcher.add_handler(reload_handler)
updater.dispatcher.add_handler(air_handler)

In [23]:
updater.start_polling(clean=True, timeout=3)
updater.idle()

/tmp/ipykernel_3684/1054514104.py:1: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  updater.start_polling(clean=True, timeout=3)


Unauthorized: Unauthorized